# Set Up

## Colab Installation

Run the following code block if you are using this Jupyter notebook in Google Colab. Otherwise skip to importing the packages.

In [ ]:
# Run these statements if you are using Google Colab. Otherwise skip this step.
!sudo apt install cm-super dvipng texlive-latex-extra texlive-latex-recommended
!sudo apt install cm-super
!pip install git+https://github.com/QMCSoftware/QMCSoftware.git@SURE2024

## Importing Packages

In [ ]:
import matplotlib.pyplot as plt
import math
import numpy as np
import scipy as sp
import qmcpy as qp
from scipy import stats
from copy import deepcopy
import sympy as sy
import sympy.stats
# figpath = '' #this path sends the figures to the directory that you want
# savefigs = True  #save the figures on your disk if True
# imgfrmt = 'pdf' #kind of format to save figures as

# Definitions of Python Functions

The following functions are things that will be repeatedly used in the notebook so that there isn't duplicate code all over the place.

## Graphing Functions

The following functions are to help with graphing.

In [ ]:
def make_grid(d=3, nper=2):
    """this function creates a nper^d x d matrix of grid points"""
    xpts = np.linspace(1/(2*nper), 1.0-1/(2*nper), nper)  # array of coordinate values
    xgrid = xpts  # initial grid
    n = nper  # initial length of grid
    for ii in range(1,d):
        xgrid0 = xgrid  # store the old grid
        xgrid = np.vstack((xgrid0, xpts[0] * np.ones((1, n))))  # stack the next coordinate
        for jj in range(1, nper):
            xgrid = np.hstack((xgrid, np.vstack((xgrid0, xpts[jj] * np.ones((1, n))))))
        n = n * nper  # update the length of the grid
    return xgrid.T  # take the transpose to that coordinates are columns

def plot_grid(xgrid, coord_to_plot=[0, 1], savefig=True, figpath=''):
    """Plots a dot grid."""
    n = xgrid.shape[0]
    d = xgrid.shape[1]
    coord_to_plot = np.minimum(coord_to_plot, [d, d]), # ensure that they are no more than d
    coord_to_plot = coord_to_plot[0]
    coord_to_plot = np.maximum(coord_to_plot, [1, 1]), # ensure that they are no less than 1
    coord_to_plot = coord_to_plot[0]
    fig, ax = plt.subplots()
    ax.plot(xgrid[:,coord_to_plot[0]-1], xgrid[:,coord_to_plot[1]-1], 'b.', markersize = 10, zorder=10, clip_on=False)
    ax.set_title(f"{n:.0f} Grid Points for $d = ${d:.0f}")
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    ax.set_xlabel(r'$x_{i%d}$'%(coord_to_plot[0]), fontsize=14)
    ax.set_ylabel(r'$x_{i%d}$'%(coord_to_plot[1]), fontsize=14)
    ax.set_aspect(1)
    if savefigs:
        fig.savefig(figpath + f"{n:.0f}gridpts_d{d:.0f}." + imgfrmt, format=imgfrmt)


In [ ]:
def graph_kernel(bandwidths, lin_spaces, kernel, ylim_lst=[-0.05, 1.2]):
  """Graphs a given kernel, with given bandwidths, lin_spaces
  bandwidths = a list of bandwidths
  lin_spaces = a list of 3 integers: start, finish, and number of points"""
  ypts = np.linspace(lin_spaces[0], lin_spaces[1], lin_spaces[2])
  nband = len(bandwidths)
  fig, ax = plt.subplots(1, 3, figsize=(5*nband, 5))
  for ii in range(nband):
      ax[ii].plot(ypts,kernel(ypts/bandwidths[ii]) / bandwidths[ii])
      ax[ii].grid(False)
      ax[ii].set_ylim(ylim_lst)
      ax[ii].set_xlabel(r'$y$', fontsize=20)
      ax[ii].set_ylabel(r'$h = {%0.1f}$'%bandwidths[ii], fontsize=20)
    #   if (ii == 0):
    #       ax[ii].set_ylabel(r'$\widetilde{k}(y) = \exp(-(y/h)^2)/(\sqrt{\pi}h), \ h = {%0.1f}$'%bandwidths[ii], fontsize=20)
    #   else:
    #       ax[ii].set_ylabel(r'$h = {%0.1f}$'%bandwidths[ii], fontsize=20)

In [ ]:
def generate_sobol_iid_graph(bandwidths, lin_spaces, kde, kernel, yiid, ysob, ylim_lst=[-0.05, 1.2]):
  """Generates a graph using IID and Sobol sample points.
  bandwidths = a list of bandwidths
  lin_spaces = a list of 3 integers: start, finish, and number of points
  kde = name of kde function
  kernel = name of kernel function
  yiid = set of iid points
  ysob = set of LD points generated with sobol sequence"""
  nband = len(bandwidths)
  yeval = np.linspace(lin_spaces[0], lin_spaces[1], lin_spaces[2])
  for ii in range(nband):
      rhovaliid = kde(kernel, yiid, bandwidths[ii], yeval)
      rhovalsob = kde(kernel, ysob, bandwidths[ii], yeval)
      fig,ax = plt.subplots()
      ax.plot(yeval, rhovaliid, label = "IID")
      ax.plot(yeval, rhovalsob, label = "Sobol")
      ax.legend()
      ax.set_ylim(ylim_lst)
      ax.set_title(f"$h = {bandwidths[ii]:.2f}$")

In [ ]:
def generate_graphs(bandwidths, lin_spaces, kde, kernel, sample_points, ylim_lst=[-0.05, 1.2]):
  """
  Generate graphs involving a kde, kernel, and sample_points.
  sample_points is a list of IID, Sobol, Lattice, DNB2, and Halton points.
  """
  nband = len(bandwidths)
  yeval = np.linspace(lin_spaces[0], lin_spaces[1], lin_spaces[2])
  rho_list = list()
  for ii in range(nband):
      rhovaliid = kde(kernel, sample_points[0], bandwidths[ii], yeval)
      rhovalsob = kde(kernel, sample_points[1], bandwidths[ii], yeval)
      rhovallattice = kde(kernel, sample_points[2], bandwidths[ii], yeval)
      rhovaldnetb2 = kde(kernel, sample_points[3], bandwidths[ii], yeval)
      rhovalhalton = kde(kernel, sample_points[4], bandwidths[ii], yeval)
      fig, ax = plt.subplots()
      ax.plot(yeval, rhovaliid, label = "IID")
      ax.plot(yeval, rhovalsob, label = "Sobol")
      ax.plot(yeval, rhovallattice, label = "Lattice")
      ax.plot(yeval, rhovaldnetb2, label = "Digital Net B2")
      ax.plot(yeval, rhovalhalton, label = "Halton")
      ax.legend()
      ax.set_ylim(ylim_lst)
      ax.set_title(f"$h = {bandwidths[ii]:.2f}$")

## Generate Functions

The following functions are used to generate different types of sample points.

In [ ]:
def gen_iid_and_sobol(dimen = 3, n = 2**12):
  """Generate IID and Sobol sample points based on given sample size and dimensions."""
  coord_wts = 2.0 ** (-np.array(range(0, dimen)))
  iidpts = qp.IIDStdUniform(dimen).gen_samples(n)
  sobpts = qp.Sobol(dimen).gen_samples(n)
  return (iidpts, sobpts)

def gen_iid_ld_pts(dimen = 3, n = 2**12):
  """
  Generate IID, Sobol, Lattic, DNB2, and Halton sample points,
  based on dimension and smaple size.
  """
  coord_wts = 2.0**(-np.array(range(0, dimen)))
  iidpts = qp.IIDStdUniform(dimen).gen_samples(n)
  sobpts = qp.Sobol(dimen).gen_samples(n)
  latticepts = qp.Lattice(dimen).gen_samples(n)
  dnetb2pts = qp.DigitalNetB2(dimen).gen_samples(n)
  haltonpts = qp.Halton(dimen).gen_samples(n)
  return (iidpts, sobpts, latticepts, dnetb2pts, haltonpts)

## Generating Points

Generating sequences of IID and LD points, with dimensions 1, 2, 3, and 5.

In [ ]:
iidpts1_10, sobpts1_10 = gen_iid_and_sobol(dimen = 1, n = 2**10)
iidpts1_11, sobpts1_11 = gen_iid_and_sobol(dimen = 1, n = 2**11)
iidpts1_12, sobpts1_12 = gen_iid_and_sobol(dimen = 1, n = 2**12)
iidpts1_13, sobpts1_13 = gen_iid_and_sobol(dimen = 1, n = 2**13)
iidpts1_14, sobpts1_14 = gen_iid_and_sobol(dimen = 1, n = 2**14)

iidpts2_10, sobpts2_10 = gen_iid_and_sobol(dimen = 2, n = 2**10)
iidpts2_11, sobpts2_11 = gen_iid_and_sobol(dimen = 2, n = 2**11)
iidpts2_12, sobpts2_12 = gen_iid_and_sobol(dimen = 2, n = 2**12)
iidpts2_13, sobpts2_13 = gen_iid_and_sobol(dimen = 2, n = 2**13)
iidpts2_14, sobpts2_14 = gen_iid_and_sobol(dimen = 2, n = 2**14)

iidpts3_10, sobpts3_10 = gen_iid_and_sobol(dimen = 3, n = 2**10)
iidpts3_11, sobpts3_11 = gen_iid_and_sobol(dimen = 3, n = 2**11)
iidpts3_12, sobpts3_12 = gen_iid_and_sobol(dimen = 3, n = 2**12)
iidpts3_13, sobpts3_13 = gen_iid_and_sobol(dimen = 3, n = 2**13)
iidpts3_14, sobpts3_14 = gen_iid_and_sobol(dimen = 3, n = 2**14)

iidpts5_10, sobpts5_10 = gen_iid_and_sobol(dimen = 5, n = 2**10)
iidpts5_11, sobpts5_11 = gen_iid_and_sobol(dimen = 5, n = 2**11)
iidpts5_12, sobpts5_12 = gen_iid_and_sobol(dimen = 5, n = 2**12)
iidpts5_13, sobpts5_13 = gen_iid_and_sobol(dimen = 5, n = 2**13)
iidpts5_14, sobpts5_14 = gen_iid_and_sobol(dimen = 5, n = 2**14)

In [ ]:
# dim = 1, n = 2**12
# iidpts1, sobpts1, latticepts1, dnetb2pts1, haltonpts1 = gen_iid_ld_pts(dimen = 1, n = 2**12)
# print("iid: ", iidpts1)
# print("sobol: ", sobpts1)
# print("lattice: ", latticepts1)
# print("digital net B2: ", dnetb2pts1)
# print("halton: ", haltonpts1)

## Kernels and KDEs

The following are definitions of different kernels used in the notebook. Some may be defined later, but these are ones used throughout the notebook or commonly used/known.

The gaussian kernel defined below represents the following:

$$
\frac{e^{-y^2}}{\sqrt{\pi}}
$$

In [ ]:
def kernel(y):
    return np.exp(-y * y) / np.sqrt(np.pi)

#### Generate some $y_i=f(x_i)$ points
$
f(\mathbf{x}) = 10 \exp(- w_1x_1 - \cdots - w_d x_d) \sin(w_1x_1 + \cdots +w_d x_d)
$

In [ ]:
def testfun1(x, coord_wts = 1): # function f(x) defines the random variable
    wtx = np.multiply(x, coord_wts)
    y = 10 * np.exp(-wtx.sum(1)) * np.sin(np.pi * wtx.sum(1))
    #y = x[:,0] # if x is uniform, then y is uniform
    return y

The following *Kernel Density Estimator* (KDE)

$$
\varrho() = \frac{k\left(\frac{y - Y}{h}\right)}{h}
$$

*(Still needs work.)*

In [ ]:
def kde(kernel, ypts, bandwidth, yeval):
    neval = len(yeval)
    rhoval = np.zeros_like(yeval) #values of density estimator
    for ii in range(len(yeval)):
        rhoval[ii] = np.mean(kernel((yeval[ii]-ypts) / bandwidth)) / bandwidth
    return rhoval

# Kernel Density Estimator for Uniform Distribution

In [ ]:
def testfun_unif(x, coord_wts = 1): # function f(x) defines the random variable
    wtx = np.multiply(x, coord_wts)
    # y = 10*np.exp(-wtx.sum(1))*np.sin(np.pi*wtx.sum(1))
    y = x[:,0] # if x is uniform, then y is uniform
    return y

# Testing Different Kernels 
(http://dx.doi.org/10.4236/ojapps.2013.31B012)

The following Python function definition is the expression:

$$
y=\begin{cases}
            \frac{3}{4}(1-y^{2}), & |y| \leq 1\\
            0, & Otherwise
        \end{cases}
$$

In [ ]:
# Epanechnikov (parabolic)--best performance, highest efficiency according to paper
def kernel_epan(y):
    return np.heaviside(1-np.absolute(y), 1/2)*(0.75*(1-(y**2)))

The following Python function definition is the expression:

$$
y=\begin{cases}
            \frac{\pi}{4}\cos\left(\frac{\pi}{2} y\right), & |y| \leq 1\\
            0, & Otherwise
        \end{cases}
$$

In [ ]:
# cosine--has good performance according to paper?
def kernel_cos(y):
    return np.heaviside(1-np.absolute(y), 1/2)*(math.pi/4)*np.cos((math.pi/2)*y)

The following Python function defines the kernel:

$$
y=\begin{cases}
            1 - |y|, & |y| \leq 1\\
            0, & Otherwise
        \end{cases}
$$

In [ ]:
# triangular kernel
def kernel_tri(y):
    return np.heaviside(1-np.absolute(y), 1/2)*(1 - np.absolute(y))

The following Python function defines the kernel:

$$
y=\begin{cases}
            \frac{1}{\pi} (1-\frac{y^2}{2}) e^-(\frac{y^2}{2}), & |y| \leq 1\\
            0, & Otherwise
        \end{cases}
$$

In [ ]:
# laplacian of Gaussian--performed poorly according to paper
def kernel_laog(y):
    return np.heaviside(1-np.absolute(y), 1/2)*((1/math.pi)*(1-(y**2)/2)*np.exp(-(y**2)/2))

## Hermite Kernel

The following comes from Dr. Hickernell's KDE experiment notebook.


In [ ]:
# Original hermite kernel function from Dr. Hickernell
y = sy.symbols('y', real = True)
h = sy.symbols('h', positive = True)
def sy_hermite_kernel(y = sy.symbols('y', real = True), band_width = sy.symbols('h', positive = True), r = 1, only_coef = False):
    '''
    band_width = the kernel band width, often denoted h; a smaller bandwidth means a peakier kernel
    r = number of terms in our kernel
    '''
    
#    import sympy as sy
    wt = sy.exp(-y*y) #normalizing weight for Hermite functions
    k = sy.zeros(r,1) #initialize a vector of bases that make up the kernel
    intk = sy.zeros(r,r) #initialize a matrix of integrals of the bases with respect to even degree polynomials
    for ii in range(r):
        k[ii] = sy.hermite(2*ii,y)*wt #form the bases
        for jj in range(r):
            intk[jj,ii] = sy.integrate(k[ii]*(y**(2*jj)),(y,-sy.oo,sy.oo)) #evaluate the integrals against even degree monomials
            # to hardcode the above line:
            # r = 0: intk[jj, ii] = (np.exp(-y**2))/((math.pi)**0.5)
            # r = 1: intk[jj, ii] = ((3-2*y**2)*np.exp(-y**2))/(2*(math.pi)**0.5)
            # r = 2: intk[jj, ii] = ((4*y**4 - 20*y**2 +15)*np.exp(-y**2))/(8*(math.pi)**0.5)
    tot_int = sy.Matrix.vstack(sy.ones(1,1),sy.zeros(r-1,1)) #the kernel should integrate to one, 
        #and the kernel integrated against y, y^2, ..., y^r should be zero
#    print(tot_int)
    coef = sy.Matrix(list(sy.linsolve((intk,tot_int)))) #solve the system of equations for the coefficients of the bases
    if only_coef:
        return coef
    print(coef)
#    ktilde = coef.dot(k)
#    integk = sy.integrate(ktilde,(y,-sy.oo,sy.oo))
#    print('integk =',integk)
    ktilde = coef.dot(k.subs(y,y/band_width)/band_width) #form the isotropic kernel and insert the bandwidth dependency
#    integ = sy.integrate(ktilde,(y,-sy.oo,sy.oo))
#    print('integ =',integ)
#    print(band_width)
#    print(ktilde)
    return coef.dot(k.subs(y,y/band_width)/band_width)

In [ ]:
# testing original hermite kernel function

test_kernel_sy = sy_hermite_kernel(r=1) # r = number terms in kernel
print(type(test_kernel_sy))
print('kernel =',test_kernel_sy)
integ = sy.integrate(test_kernel_sy,(y,-sy.oo,sy.oo))
print('integ =',integ) # should be 1
test_kernel_sy = sy_hermite_kernel(r=1,only_coef = True)
print(test_kernel_sy)
test_kernel_sy = sy_hermite_kernel(r=2,only_coef = True)
print(test_kernel_sy)
test_kernel_sy = sy_hermite_kernel(r=3,only_coef = True)
print(test_kernel_sy)

In [ ]:
# hardcoding hermite kernel coefficients

def hardcode_hermite_kernel(r):
    '''
    returns hermite coefficients for r = 0, 1, or 2
    band_width = the kernel band width, often denoted h; a smaller bandwidth means a peakier kernel
    r = number of terms in our kernel
    '''
    if r==0:
        return sy.Matrix([[1/(math.pi)**0.5]])
    elif r==1:
        return sy.Matrix([[1/(math.pi)**0.5, -1/(4*(math.pi)**0.5)]])
    elif r==2:
        return sy.Matrix([[1/(math.pi)**0.5, -1/(4*(math.pi)**0.5), 1/(32*(math.pi)**0.5)]])
    else:
        raise ValueError("Invalid input. r must be 0, 1, or 2.")
    
# Coefficients: 
# r = 0 (corresponds to r = 1 from original function): Matrix([[1/sqrt(pi)]])
# r = 1 (corresponds to r = 2 from original function): Matrix([[1/sqrt(pi), -1/(4*sqrt(pi))]])
# r = 2 (correpsonds to r = 3 from original function): Matrix([[1/sqrt(pi), -1/(4*sqrt(pi)), 1/(32*sqrt(pi))]])

In [ ]:
# hardcoding hermite kernel coefficients without sympy

def hardcode_hermite_kernel_simple(r):
    '''
    returns hermite coefficients for r = 0, 1, or 2
    band_width = the kernel band width, often denoted h; a smaller bandwidth means a peakier kernel
    r = number of terms in our kernel
    '''
    if r==0:
        return np.matrix([1/(math.pi)**0.5])
    elif r==1:
        return np.matrix([[1/(math.pi)**0.5, -1/(4*(math.pi)**0.5)]])
    elif r==2:
        return np.matrix([[1/(math.pi)**0.5, -1/(4*(math.pi)**0.5), 1/(32*(math.pi)**0.5)]])
    else:
        raise ValueError("Invalid input. r must be 0, 1, or 2.")
    
# Coefficients: 
# r = 0 (corresponds to r = 1 from original function): Matrix([[1/sqrt(pi)]])
# r = 1 (corresponds to r = 2 from original function): Matrix([[1/sqrt(pi), -1/(4*sqrt(pi))]])
# r = 2 (correpsonds to r = 3 from original function): Matrix([[1/sqrt(pi), -1/(4*sqrt(pi)), 1/(32*sqrt(pi))]])

In [ ]:
# original function: define hermite kernel for r = 1 (corresopnds to r = 0 in the hardcoded case)
def hermite_kernel(y, r = 1):
    '''
    r = number of terms in our kernel
    '''
    coef = np.array(sy.N(sy_hermite_kernel(r = r, only_coef = True)))
    # print(coef)
    poly = sp.special.hermite(0)
    # print(poly)
    k = sp.special.hermite(0)(y)*coef[0] #initialize a vector of kernel values
    # print(k)
    for ii in range(1,r):
        k += sp.special.hermite(2*ii)(y)*coef[ii] #add the additional terms
    k*=np.exp(-y*y/2) #normalizing weight for Hermite functions #form the isotropic kernel and insert the bandwidth dependency
    return k

In [ ]:
# hardcoded: define hermite kernel for r = 0 (corresponds to r = 1 in the original case)
def hardcode_hermite_kernel_weight(y, r = 0):
    '''
    r = number of terms in our kernel
    '''
    coef = np.array((hardcode_hermite_kernel(r = r)))
    # print(coef)
    # poly = sp.special.hermite(0)
    # print(poly)
    k = sp.special.hermite(0)(y)*coef[0][0] #initialize a vector of kernel values
    # print(k)
    for ii in range(0,r):
        k += sp.special.hermite(2*ii)(y)*coef[0][ii] #add the additional terms
    k*=np.exp(-y*y/2) #normalizing weight for Hermite functions #form the isotropic kernel and insert the bandwidth dependency
    return k

In [ ]:
# hardcoded without scipy: define hermite kernel for r = 0 (corresponds to r = 1 in the original case)
def simple_hardcode_hermite_kernel_weight(y, r = 0):
    '''
    r = number of terms in our kernel
    '''
    coef = np.array((hardcode_hermite_kernel(r = r)))
    hermite_polys = [sp.poly1d([1.]), sp.poly1d([4.,  0., -2.]), sp.poly1d([ 1.6000000e+01,  0.0000000e+00, -4.8000000e+01, -8.8817842e-16,
        1.2000000e+01])]
    # print(coef)
    # poly = sp.special.hermite(0)
    # print(poly)
    k = sp.poly1d([1.])(y)*coef[0][0] #initialize a vector of kernel values
    # print(k)
    for ii in range(0,r):
        k += hermite_polys[r](y)*coef[0][ii] #add the additional terms
    k*=np.exp(-y*y/2) #normalizing weight for Hermite functions #form the isotropic kernel and insert the bandwidth dependency
    return k

In [ ]:
# original: define hermite kernel for r = 2 (corresopnds to r = 1 in the hardcoded case)
def hermite_kernel_2(y, r = 2):
    '''
    r = number of terms in our kernel
    '''
    coef = np.array(sy.N(sy_hermite_kernel(r = r, only_coef = True)))
    # print(coef)
    poly = sp.special.hermite(0) # why use the 0th polynomial?
    # print(poly)
    # for i in range(len(coef[0])):
    #     print(coef[0][i])
    k = sp.special.hermite(0)(y)*coef[0][0] #initialize a vector of kernel values
    # print(k)
    for ii in range(1,r):
        k += sp.special.hermite(2*ii)(y)*coef[0][ii] #add the additional terms
    k*=np.exp(-y*y/2) #normalizing weight for Hermite functions #form the isotropic kernel and insert the bandwidth dependency
    return k

In [ ]:
# hardcoded: define hermite kernel for r = 1 (corresponds to r = 2 in the original case)
def hardcode_hermite_kernel_weight_1(y, r = 1):
    '''
    r = number of terms in our kernel
    '''
    coef = np.array((hardcode_hermite_kernel(r = r)))
    # print(coef)
    poly = sp.special.hermite(0)
    # print(poly)
    k = sp.special.hermite(0)(y)*coef[0][0] #initialize a vector of kernel values
    # print(k)
    for ii in range(0,r):
        k += sp.special.hermite(2*ii)(y)*coef[0][ii] #add the additional terms
    k*=np.exp(-y*y/2) #normalizing weight for Hermite functions #form the isotropic kernel and insert the bandwidth dependency
    return k

In [ ]:
# hardcoded without scipy: define hermite kernel for r = 1 (corresponds to r = 2 in the original case)
def simple_hardcode_hermite_kernel_weight_1(y, r = 1):
    '''
    r = number of terms in our kernel
    '''
    coef = np.array((hardcode_hermite_kernel(r = r)))
    hermite_polys = [sp.poly1d([1.]), sp.poly1d([4.,  0., -2.]), sp.poly1d([ 1.6000000e+01,  0.0000000e+00, -4.8000000e+01, -8.8817842e-16,
        1.2000000e+01])]
    # print(coef)
    # poly = sp.special.hermite(0)
    # print(poly)
    k = sp.poly1d([1.])(y)*coef[0][0] #initialize a vector of kernel values
    # print(k)
    for ii in range(0,r):
        k += hermite_polys[r](y)*coef[0][ii] #add the additional terms
    k*=np.exp(-y*y/2) #normalizing weight for Hermite functions #form the isotropic kernel and insert the bandwidth dependency
    return k

In [ ]:
# hardcoded: define hermite kernel for r = 2 (corresponds to r = 3 in the original case)
def hardcode_hermite_kernel_weight_2(y, r = 2):
    '''
    r = number of terms in our kernel
    '''
    coef = np.array((hardcode_hermite_kernel(r = r)))
    # print(coef)
    poly = sp.special.hermite(0)
    # print(poly)
    k = sp.special.hermite(0)(y)*coef[0][0] #initialize a vector of kernel values
    # print(k)
    for ii in range(0,r):
        k += sp.special.hermite(2*ii)(y)*coef[0][ii] #add the additional terms
    k*=np.exp(-y*y/2) #normalizing weight for Hermite functions #form the isotropic kernel and insert the bandwidth dependency
    return k

In [ ]:
# hardcoded: define hermite kernel for r = 2 (corresponds to r = 3 in the original case)
def simple_hardcode_hermite_kernel_weight_2(y, r = 2):
    '''
    r = number of terms in our kernel
    '''
    coef = np.array((hardcode_hermite_kernel(r = r)))
    hermite_polys = [sp.poly1d([1.]), sp.poly1d([4.,  0., -2.]), sp.poly1d([ 1.6000000e+01,  0.0000000e+00, -4.8000000e+01, -8.8817842e-16,
        1.2000000e+01])]
    # print(coef)
    # poly = sp.special.hermite(0)
    # print(poly)
    k = sp.poly1d([1.])(y)*coef[0][0]  #initialize a vector of kernel values
    # print(k)
    for ii in range(0,r):
        k += hermite_polys[r](y)*coef[0][ii] #add the additional terms
    k*=np.exp(-y*y/2) #normalizing weight for Hermite functions #form the isotropic kernel and insert the bandwidth dependency
    return k

In [ ]:
# using simplified np.matrix for coefficients, instead of sympy.matrix
def hardcode_hermite_kernel_weight_v2(y, r = 2):
    '''
    r = number of terms in our kernel
    '''
    coef = np.array((hardcode_hermite_kernel_simple(r = r)))
    # print(coef)
    # poly = sp.special.hermite(0)
    # print(poly)
    k = sp.special.hermite(0)(y) * coef[0][0] #initialize a vector of kernel values
    # print(k)
    for ii in range(0, r):
        k += sp.special.hermite(2*ii)(y) * coef[0][ii] #add the additional terms
    k *= np.exp(-y*y/2) #normalizing weight for Hermite functions #form the isotropic kernel and insert the bandwidth dependency
    return k

## Testing Different Functions f(x)--using Gaussian kernel in all cases.

### Exponential Distribution: 

If X has a standard uniform distribution, i.e., $X \sim U(0, 1)$, then the random variable $Y = -\lambda^{-1} ln(X)$ has an exponential distribution with rate parameter $\lambda$.

In [ ]:
# exponential, lambda = 1:
def testfun_exp(x,coord_wts = 1): # function f(x) defines the random variable
    wtx = np.multiply(x,coord_wts)
    y = -np.log(wtx)
    return y

### Laplace Distribution:

If X and Y have a standard uniform distribution, i.e., $X, Y \sim U(0, 1)$, then $\log{}{\frac{X}{Y}} \sim \text{Laplace}(0, 1)$. 

In [ ]:
## laplace: mu = 0, b = 1
def testfun_laplace(x1,x2,coord_wts = 1): # function f(x) defines the random variable
    wtx1 = np.multiply(x1,coord_wts)
    wtx2 = np.multiply(x2,coord_wts)
    y = np.log10(wtx1/wtx2)
    return y

## Error Analysis:

In [ ]:
# import scipy.integrate as integrate

In [ ]:
# def true_func(true_func, x):
#     '''
#     returns the true value of a function given an input x
#     true_func = the function we are trying to estimate (e.g., f(x), exponential PDF, laplace PDF, etc.)
#     x = a float
#     '''
#     return true_func(x)

In [ ]:
# def estimated_pts(kde_func, kernel_func, points, bw, x):
#     '''
#     returns the estimation of a function at a point x using the specified KDE
#     kde_func = the function that builds the KDE
#     kernel_func = function that defines the kernel
#     points = an array of randomly-generated iid or ld points
#     bw = bandwidth (a float)
#     x = a float
#     '''
#     return kde_func(kernel_func, points, bw, x)

In [ ]:
# def squared_difference(true_func, estimate_func, x):
#     '''
#     returns the squared error between the true value and estimated value at a point x
#     true_func = actual function that we are trying to estimate (e.g., f(x), exponential PDF, laplace PDF, etc.)
#     estimate_func = function that returns estimation of the function at the point x
#     x = a float
#     '''
#     return (true_func(x) - estimate_func(x))**2

In [ ]:
# def optimal_h(h_space, pts, kde, kernel, true_function):
#     '''
#     returns optimal bandwidth and lowest value of MISE
#     h_space: a set of bandwidths to be tested (e.g., np.linspace(0.001, 3, 100))
#     pts: set of randomly generated points (IID or LD)
#     kde: kde function
#     kernel: function that defines kernel
#     true_function: function that we are trying to estimate (e.g., uniform, exponential, etc.)
#     '''

#     h_optimal = 0
#     lowest_mise = np.inf

#     for h in h_space:
#         def estimated_pts(x):
#             return kde(kernel, pts, h, x)
#         def squared_difference_pts(x):
#             return (true_function(x) - estimated_pts(x))**2
#         mise, error = integrate.quad(squared_difference_pts, -4, 8)
#         if mise < lowest_mise:
#             lowest_mise = mise
#             h_optimal = h

#     return(lowest_mise, h_optimal)

In [ ]:
import pandas as pd

In [ ]:
def graph_rmse_vs_bandwidth(h_list, rmse_func, n, pts, kernel, kde, xeval, graph_title):
    '''
    generates a graph of rmse vs. log-scaled bandwidth
    
    h_list = list of bandwidths to be tested (log-scaled bandwidth on x-axis)
    rmse_func = function that computes rmse for a particular bandwidth value
    n = number of points
    pts = set of points (IID or LD)
    kernel = kernel function
    kde = kernel density estimator function
    xeval = set of points that is range of x-axis (e.g., np.linspace(-4, 8, 500))
    graph_title = a string that will become title of graph
    '''

    rmse_list = []
    for i in range(len(h_list)):
        rmse_list.append(rmse_func(h_list[i], n, pts, kernel, kde, xeval))

    min_index = rmse_list.index(min(rmse_list))
    print("Minimum rmse = " 
        + str(rmse_list[min_index]) 
        + "\nand corresponding bandwidth is " 
        + str(h_list[min_index])
        + ".\nLog of optimal bandwidth is "
        + str(np.log(h_list[min_index])))
    
    fig, ax = plt.subplots()
    ax.plot(h_list, rmse_list)
    ax.set_title(graph_title)
    ax.set_xlabel('Log-Scaled Bandwidth')
    ax.set_ylabel('RMSE')
    ax.set_xscale('log')

In [ ]:
def weighted_graph_rmse_vs_bandwidth(h_list, rmse_func, n, pts, kernel, kde, xeval, weights, graph_title):
    '''
    generates a graph of rmse vs. log-scaled bandwidth
    
    h_list = list of bandwidths to be tested (log-scaled bandwidth on x-axis)
    rmse_func = function that computes rmse for a particular bandwidth value
    n = number of points
    pts = set of points (IID or LD)
    kernel = kernel function
    kde = kernel density estimator function
    xeval = set of points that is range of x-axis (e.g., np.linspace(-4, 8, 500))
    weights = list of weights
    graph_title = a string that will become title of graph
    '''

    rmse_list = []
    for i in range(len(h_list)):
        rmse_list.append(rmse_func(h_list[i], n, pts, kernel, kde, xeval, weights))

    min_index = rmse_list.index(min(rmse_list))
    print("Minimum rmse = " 
        + str(rmse_list[min_index]) 
        + "\nand corresponding bandwidth is " 
        + str(h_list[min_index])
        + ".\nLog of optimal bandwidth is "
        + str(np.log(h_list[min_index])))
    
    fig, ax = plt.subplots()
    ax.plot(h_list, rmse_list)
    ax.set_title(graph_title)
    ax.set_xlabel('Log-Scaled Bandwidth')
    ax.set_ylabel('RMSE')
    ax.set_xscale('log')

In [ ]:
def rmse_heatmap(h_list, n_list_power, rmse_func, pts_list, kernel, kde, xeval, heatmap_title):
    '''
    generates a heatmap with sample size on the x-axis, bandwidths on the y-axis, and rmse represented by colored boxes

    h_list = list of bandwidths (floats) to be tested
    n_list_power = list of magnitudes of sample sizes to be tested (integers; sample sizes are 2^n for all n in n_list_power)
    rmse_func = function that calculates rmse
    pts_list = a list of IID or LD points. each element in the list is a sequence of points with different sample sizes.
    kernel = kernel function
    kde = kde function
    xeval = set of points that is range of x-axis (e.g., np.linspace(-4, 8, 500))
    heatmap_title = title of heatmap (string)
    '''
    
    d = {}
    h_list_str = [str(h) for h in h_list]

    for i in range(len(n_list_power)):
        rmse_list = []
        for j in range(len(h_list)):
            rmse_list.append(rmse_func(h_list[j], 2**(n_list_power[i]), pts_list[i], kernel, kde, xeval))
        
        d[str(n_list_power[i])] = pd.Series(rmse_list, index = h_list_str)

    df = pd.DataFrame(d)
    
    fig, ax = plt.subplots(figsize=(4, 4))
    im = ax.imshow(df)
    ax.set_title(heatmap_title, size=10)
    fig.tight_layout()

    yaxis = df.index.values
    xaxis = df.columns.values
    # Add axis tick labels
    ax.set_xticks(np.arange(len(xaxis)), 
                labels=xaxis)
    ax.set_yticks(np.arange(len(yaxis)), 
                labels=yaxis)

    cbar = ax.figure.colorbar(im, 
                            ax = ax,
                            shrink=0.75)

    # Rotate the tick labels to be more legible
    plt.setp(ax.get_xticklabels(),
            rotation = 45,
            ha = "right",
            rotation_mode = "anchor")

    plt.xlabel("Sample size magnitude (powers of 2)") 
    plt.ylabel("Bandwidth") 

In [ ]:
def weighted_rmse_heatmap(h_list, n_list_power, rmse_func, pts_list, kernel, kde, xeval, weights, heatmap_title):
    '''
    generates a heatmap with sample size on the x-axis, bandwidths on the y-axis, and rmse represented by colored boxes

    h_list = list of bandwidths (floats) to be tested
    n_list_power = list of magnitudes of sample sizes to be tested (integers; sample sizes are 2^n for all n in n_list_power)
    rmse_func = function that calculates rmse
    pts_list = a list of IID or LD points. each element in the list is a sequence of points with different sample sizes.
    kernel = kernel function
    kde = kde function
    xeval = set of points that is range of x-axis (e.g., np.linspace(-4, 8, 500))
    weights = list of weights
    heatmap_title = title of heatmap (string)
    '''

    d = {}
    h_list_str = [str(h) for h in h_list]

    for i in range(len(n_list_power)):
        rmse_list = []
        for j in range(len(h_list)):
            rmse_list.append(rmse_func(h_list[j], 2**(n_list_power[i]), pts_list[i], kernel, kde, xeval, weights))
        
        d[str(n_list_power[i])] = pd.Series(rmse_list, index = h_list_str)

    df = pd.DataFrame(d)
    
    fig, ax = plt.subplots(figsize=(4, 4))
    im = ax.imshow(df)
    ax.set_title(heatmap_title, size=10)
    fig.tight_layout()

    yaxis = df.index.values
    xaxis = df.columns.values
    # Add axis tick labels
    ax.set_xticks(np.arange(len(xaxis)), 
                labels=xaxis)
    ax.set_yticks(np.arange(len(yaxis)), 
                labels=yaxis)

    cbar = ax.figure.colorbar(im, 
                            ax = ax,
                            shrink=0.75)

    # Rotate the tick labels to be more legible
    plt.setp(ax.get_xticklabels(),
            rotation = 45,
            ha = "right",
            rotation_mode = "anchor")

    plt.xlabel("Sample size magnitude (powers of 2)") 
    plt.ylabel("Bandwidth") 

In [ ]:
# def graph_optimal_bandwidth_vs_n(h_list, n_list_power, rmse_func, pts_list, kernel, kde, xeval, graph_title):
#     '''
#     generates a graph with sample size on the x-axis and optimal bandwidth on the y-axis

#     h_list = list of bandwidths (floats) to be tested. for each sample size, the bandwidth that corresponds with the lowest rmse will be plotted.
#     n_list_power = list of magnitudes of sample sizes to be tested (integers; sample sizes are 2^n for all n in n_list_power)
#     rmse_func = function that calculates rmse
#     pts_list = a list of IID or LD points. each element in the list is a sequence of points with different sample sizes.
#     kernel = kernel function
#     kde = kde function
#     xeval = set of points that is range of x-axis (e.g., np.linspace(-4, 8, 500))
#     graph_title = title of graph (string)
#     '''

#     optimal_bandwidths_dict = {}
#     rmse_values = []

#     for i in range(len(n_list_power)):
#         rmse_list = []
#         for j in range(len(h_list)):
#             rmse_list.append(rmse_func(h_list[j], 2**(n_list_power[i]), pts_list[i], kernel, kde, xeval))

#         min_index = rmse_list.index(min(rmse_list))
#         optimal_bandwidths_dict[str(n_list_power[i])] = h_list[min_index]
#         rmse_values.append(rmse_list[min_index])

#     fig, ax = plt.subplots()
#     ax.plot(list(optimal_bandwidths_dict.keys()), list(optimal_bandwidths_dict.values()))
#     ax.set_title(graph_title)
#     ax.set_xlabel('Sample size magnitude (powers of 2)')
#     ax.set_ylabel('Optimal Bandwidth')
#     # ax.set_xscale('log')

#     print("Sample size magnitudes, optimal bandwidths:")
#     print(optimal_bandwidths_dict)
#     print("Corresponding RMSE values:")
#     print(rmse_values)

In [ ]:
# def weighted_graph_optimal_bandwidth_vs_n(h_list, n_list_power, rmse_func, pts_list, kernel, kde, xeval, weights, graph_title):
#     '''
#     generates a graph with sample size on the x-axis and optimal bandwidth on the y-axis

#     h_list = list of bandwidths (floats) to be tested. for each sample size, the bandwidth that corresponds with the lowest rmse will be plotted.
#     n_list_power = list of magnitudes of sample sizes to be tested (integers; sample sizes are 2^n for all n in n_list_power)
#     rmse_func = function that calculates rmse
#     pts_list = a list of IID or LD points. each element in the list is a sequence of points with different sample sizes.
#     kernel = kernel function
#     kde = kde function
#     xeval = set of points that is range of x-axis (e.g., np.linspace(-4, 8, 500))
#     weights = list of weights
#     graph_title = title of graph (string)
#     '''

#     optimal_bandwidths_dict = {}
#     rmse_values = []

#     for i in range(len(n_list_power)):
#         rmse_list = []
#         for j in range(len(h_list)):
#             rmse_list.append(rmse_func(h_list[j], 2**(n_list_power[i]), pts_list[i], kernel, kde, xeval, weights))

#         min_index = rmse_list.index(min(rmse_list))
#         optimal_bandwidths_dict[str(n_list_power[i])] = h_list[min_index]
#         rmse_values.append(rmse_list[min_index])

#     fig, ax = plt.subplots()
#     ax.plot(list(optimal_bandwidths_dict.keys()), list(optimal_bandwidths_dict.values()))
#     ax.set_title(graph_title)
#     ax.set_xlabel('Sample size magnitude (powers of 2)')
#     ax.set_ylabel('Optimal Bandwidth')
#     # ax.set_xscale('log')

#     print("Sample size magnitudes, optimal bandwidths:")
#     print(optimal_bandwidths_dict)
#     print("Corresponding RMSE values:")
#     print(rmse_values)

### Normal distribution

In [ ]:
from scipy.stats import norm 
import numpy as np

def rmse_normal(h, n, pts, kernel, kde, xeval):
    '''
    returns rmse for a KDE that estimates a normally distributed random variable
    
    h = bandwidth
    n = number of points
    pts = set of iid or ld points
    kernel = kernel function
    xeval = set of points that is range of x-axis (e.g., np.linspace(-4, 8, 500))
    kde = kde function
    '''

    sum_errors = 0

    normal = xeval.copy()
    
    for i in range(len(normal)):
       normal[i]=norm.pdf(normal[i], loc=0, scale=1)
    
    rhoval = kde(kernel, pts, h, xeval)

    for i in range(500):
        sum_errors += (normal[i] - rhoval[i])**2
    
    rmse = np.sqrt((1/n)*sum_errors)
    return rmse

In [ ]:
#2. transforming points 
import scipy.stats as stats 

# use scipy for normal dist.--inverse CDF
def testfun_normal(x, coord_wts = 1):  #takign one uniform rv and transofmring into a normal 
    wtx= np.multiply(x, coord_wts)
    # normal_rv = stats.norm.pdf(wtx)
    normal_rv = norm.ppf(wtx)
    return normal_rv

In [ ]:
yiid1_10_normal=testfun_normal(iidpts1_10)
yiid1_11_normal=testfun_normal(iidpts1_11)
yiid1_12_normal=testfun_normal(iidpts1_12)
yiid1_13_normal=testfun_normal(iidpts1_13)
yiid1_14_normal=testfun_normal(iidpts1_14)

ysob1_10_normal=testfun_normal(sobpts1_10)
ysob1_11_normal=testfun_normal(sobpts1_11)
ysob1_12_normal=testfun_normal(sobpts1_12)
ysob1_13_normal=testfun_normal(sobpts1_13)
ysob1_14_normal=testfun_normal(sobpts1_14)

In [ ]:
xeval = np.linspace(-4, 8, 500)
print(rmse_normal(0.1, 2**10, yiid1_10_normal, kernel, kde, xeval))

In [ ]:
# h_list = np.linspace(0.001, 1, 100)
h_list = 2**np.linspace(-10, 0, 100)
graph_rmse_vs_bandwidth(h_list, rmse_normal, 2**14, ysob1_14_normal, kernel_epan, kde, xeval, "RMSE vs. bandwidth for Sobol pts: n = 2^12, dim = 1, Epan. kernel")

In [ ]:
h_list = [0.001, 0.002, 0.004, 0.01, 0.02, 0.04, 0.1, 0.2, 0.4, 1]
n_list_power = [10, 11, 12, 13, 14]
pts_list = [ysob1_10_normal, ysob1_11_normal, ysob1_12_normal, ysob1_13_normal, ysob1_14_normal]
rmse_heatmap(h_list, n_list_power, rmse_normal, pts_list, kernel_epan, kde, xeval, "RMSE for 1-D Gaussian, Epan. kernel, Sobol points")

In [ ]:
# graph_optimal_bandwidth_vs_n(h_list, n_list_power, rmse_normal, pts_list, kernel, kde, xeval, "Optimal h vs. n for 1-D Gaussian, dim = 1, Gaussian kernel, IID points")

### Uniformly distributed random variable

In [ ]:
def rmse_unif(h, n, pts, kernel, kde, xeval):
    '''
    returns rmse for a KDE that estimates a uniformly distributed random variable
    
    h = bandwidth
    n = number of points
    pts = set of iid or ld points
    kernel = kernel function
    kde = kde function
    xeval = set of points that is range of x-axis (e.g., np.linspace(-4, 8, 500))
    '''

    sum_errors = 0

    unif = xeval.copy()

    for i in range(len(unif)):
        if 0 < unif[i] < 1:
            unif[i] = 1
        else:
            unif[i] = 0
    
    rhoval = kde(kernel, pts, h, xeval)

    for i in range(500):
        sum_errors += (unif[i] - rhoval[i])**2
    
    rmse = np.sqrt((1/n)*sum_errors)
    return rmse

In [ ]:
yiid1_10_unif=testfun_unif(iidpts1_10)
yiid1_11_unif=testfun_unif(iidpts1_11)
yiid1_12_unif=testfun_unif(iidpts1_12)
yiid1_13_unif=testfun_unif(iidpts1_13)
yiid1_14_unif=testfun_unif(iidpts1_14)

ysob1_10_unif=testfun_unif(sobpts1_10)
ysob1_11_unif=testfun_unif(sobpts1_11)
ysob1_12_unif=testfun_unif(sobpts1_12)
ysob1_13_unif=testfun_unif(sobpts1_13)
ysob1_14_unif=testfun_unif(sobpts1_14)

In [ ]:
xeval = np.linspace(-4, 8, 500)
print(rmse_unif(0.1, 2**12, ysob1_10_unif, kernel, kde, xeval))

In [ ]:
# h_list = np.linspace(0.001, 1, 100)
h_list = 2**np.linspace(-10, 0, 100)
graph_rmse_vs_bandwidth(h_list, rmse_unif, 2**14, yiid1_14_unif, kernel_epan, kde, xeval, "RMSE vs. bandwidth for IID pts: 1-D Unif, n = 2^12, Epan. kernel")

In [ ]:
h_list = [0.001, 0.002, 0.004, 0.01, 0.02, 0.04, 0.1, 0.2, 0.4, 1]
n_list_power = [10, 11, 12, 13, 14]
pts_list = [yiid1_10_unif, yiid1_11_unif, yiid1_12_unif, yiid1_13_unif, yiid1_14_unif]
rmse_heatmap(h_list, n_list_power, rmse_unif, pts_list, kernel_epan, kde, xeval, "RMSE for 1-D Unif, Epan. kernel, IID points")

In [ ]:
# graph_optimal_bandwidth_vs_n(h_list, n_list_power, rmse_unif, pts_list, kernel, kde, xeval, "Optimal h vs. n for 1-D Unif, dim = 1, IID points, Epan. kernel")

### Sum of 2 uniform random variables (triangular)

In [ ]:
def rmse_triangular(h, n, pts, kernel, kde, xeval):
    '''
    returns rmse for a KDE that estimates a random variable that is the sum of 2 uniform rv's (follows a triangular distribution)
    
    h = bandwidth
    n = number of points
    pts = set of iid or ld points
    kernel = kernel function
    kde = kde function
    xeval = set of points that is range of x-axis (e.g., np.linspace(-4, 8, 500))
    '''

    sum_errors = 0

    triangular = xeval.copy()
    
    for i in range(len(triangular)):
        if 0 < triangular[i] < 1:
            triangular[i] = triangular[i]
        elif 1 <= triangular[i] < 2:
            triangular[i] = 2 - triangular[i]
        else:
            triangular[i] = 0
    
    rhoval = kde(kernel, pts, h, xeval)

    for i in range(500):
        sum_errors += (triangular[i] - rhoval[i])**2
    
    rmse = np.sqrt((1/n)*sum_errors)
    return rmse

In [ ]:
def testfun_triangular(x, coord_wts = 1): # takes a 2D sequence of IID or LD points and transforms into triangular (sum of uniforms)
    y = np.sum(x*coord_wts, axis = 1)
    return y

In [ ]:
# # testing testfun_triangular
# x = np.array([[0.2, 0.3], [0.1, 0.2]])
# print(testfun_triangular(x))

In [ ]:
# def testfun_triangular(x, coord_wts = 1):  #takign one uniform rv and transforming into triangular 
#     wtx= np.multiply(x, coord_wts)

#     # Parameters for the triangular distribution
#     a = 0  # lower limit
#     c = 1  # mode
#     b = 2  # upper limit

#     # Apply the inverse CDF method for the triangular distribution
#     def inverse_triangular_cdf(pts, a, c, b):
#         F_c = (c - a) / (b - a)
#         return np.where(pts < F_c,
#                         a + np.sqrt(pts * (b - a) * (c - a)),
#                         b - np.sqrt((1 - pts) * (b - a) * (b - c)))

#     triangular_rv = inverse_triangular_cdf(wtx, a, c, b)
#     return triangular_rv

In [ ]:
yiid2_10_triang=testfun_triangular(iidpts2_10)
yiid2_11_triang=testfun_triangular(iidpts2_11)
yiid2_12_triang=testfun_triangular(iidpts2_12)
yiid2_13_triang=testfun_triangular(iidpts2_13)
yiid2_14_triang=testfun_triangular(iidpts2_14)

ysob2_10_triang=testfun_triangular(sobpts2_10)
ysob2_11_triang=testfun_triangular(sobpts2_11)
ysob2_12_triang=testfun_triangular(sobpts2_12)
ysob2_13_triang=testfun_triangular(sobpts2_13)
ysob2_14_triang=testfun_triangular(sobpts2_14)

In [ ]:
xeval = np.linspace(-4, 8, 500)
print(rmse_triangular(0.1, 2**10, yiid2_10_triang, kernel, kde, xeval))

In [ ]:
# h_list = np.linspace(0.001, 1, 100)
h_list = 2**np.linspace(-10, 0, 100)
graph_rmse_vs_bandwidth(h_list, rmse_triangular, 2**14, yiid2_14_triang, kernel_epan, kde, xeval, "RMSE vs. bandwidth for sum of 2 unifs: IID points, n = 2^12, Epan. kernel")

In [ ]:
h_list = [0.001, 0.002, 0.004, 0.01, 0.02, 0.04, 0.1, 0.2, 0.4, 1]
n_list_power = [10, 11, 12, 13, 14]
pts_list = [yiid2_10_triang, yiid2_11_triang, yiid2_12_triang, yiid2_13_triang, yiid2_14_triang]
rmse_heatmap(h_list, n_list_power, rmse_triangular, pts_list, kernel_epan, kde, xeval, "RMSE for sum of 2 unifs, IID points, Epan. kernel")

In [ ]:
# graph_optimal_bandwidth_vs_n(h_list, n_list_power, rmse_triangular, pts_list, kernel, kde, xeval, "Optimal h vs. n for triangular distribution, dim = 1, Sobol points")

### Exponential Distribution

In [ ]:
from scipy.stats import expon

In [ ]:
def rmse_exp(h, n, pts, kernel, kde, xeval):
    '''
    returns rmse for a KDE that estimates a random variable with an exponential distribution
    
    h = bandwidth
    n = number of points
    pts = set of iid or ld points
    kernel = kernel function
    kde = kde function
    xeval = set of points that is range of x-axis (e.g., np.linspace(-4, 8, 500))
    '''

    sum_errors = 0

    exp = xeval.copy()

    for i in range(len(exp)):
        exp[i] = expon.pdf(exp[i])
    
    rhoval = kde(kernel, pts, h, xeval)

    for i in range(500):
        sum_errors += (exp[i] - rhoval[i])**2
    
    rmse = np.sqrt((1/n)*sum_errors)
    return rmse

In [ ]:
yiid1_10_exp=testfun_exp(iidpts1_10)
yiid1_11_exp=testfun_exp(iidpts1_11)
yiid1_12_exp=testfun_exp(iidpts1_12)
yiid1_13_exp=testfun_exp(iidpts1_13)
yiid1_14_exp=testfun_exp(iidpts1_14)

ysob1_10_exp=testfun_exp(sobpts1_10)
ysob1_11_exp=testfun_exp(sobpts1_11)
ysob1_12_exp=testfun_exp(sobpts1_12)
ysob1_13_exp=testfun_exp(sobpts1_13)
ysob1_14_exp=testfun_exp(sobpts1_14)

In [ ]:
xeval = np.linspace(-4, 8, 500)
print(rmse_exp(0.1, 2**12, yiid1_10_exp, kernel_epan, kde, xeval))

In [ ]:
# h_list = np.linspace(0.001, 1, 100)
h_list = 2**np.linspace(-10, 0, 100)
graph_rmse_vs_bandwidth(h_list, rmse_exp, 2**14, yiid1_14_exp, kernel_epan, kde, xeval, "RMSE vs. bandwidth for 1-D exp, IID points, n = 2^12, Epan. kernel)")

In [ ]:
h_list = [0.001, 0.002, 0.004, 0.01, 0.02, 0.04, 0.1, 0.2, 0.4, 1]
n_list_power = [10, 11, 12, 13, 14]
pts_list = [yiid1_10_exp, yiid1_11_exp, yiid1_12_exp, yiid1_13_exp, yiid1_14_exp]
rmse_heatmap(h_list, n_list_power, rmse_exp, pts_list, kernel_epan, kde, xeval, "RMSE for exp. distribution, IID points, Epan. kernel")

In [ ]:
# graph_optimal_bandwidth_vs_n(h_list, n_list_power, rmse_exp, pts_list, kernel, kde, xeval, "Optimal h vs. n for exponential distribution, Sobol points")

### Laplace Distribution

The inverse CDF of the Laplace distribution with location parameter 𝜇 and scale parameter 𝑏 is given by:
$$
F^{-1}(x) = \text{𝜇} - \text{𝑏} \cdot \text{sign}(x-0.5) \cdot \text{ln}(1-2|x-0.5|)
$$

where $x$ is a uniform random variable in the interval $[0,1]$.

In [ ]:
def rmse_laplace(h, n, pts, kernel, kde, xeval):
    '''
    returns rmse for a KDE that estimates a random variable with a laplace distribution

    h = bandwidth
    n = number of points
    pts = set of iid or ld points
    kernel = kernel function
    kde = kde function
    xeval = set of points that is range of x-axis (e.g., np.linspace(-4, 8, 500))
    '''

    sum_errors = 0

    laplace = xeval.copy()
    
    # Define location (mean) and scale parameters
    loc = 0
    scale = 1.0
 
    # Create a Laplace distribution object
    laplace_dist = stats.laplace(loc, scale)

    for i in range(len(laplace)):
        laplace[i] = laplace_dist.pdf(laplace[i])
    
    rhoval = kde(kernel, pts, h, xeval)

    for i in range(500):
        sum_errors += (laplace[i] - rhoval[i])**2
    
    rmse = np.sqrt((1/n)*sum_errors)
    return rmse

In [ ]:
def testfun_laplace(x, coord_wts = 1):  #takign one uniform rv and transforming into a laplace 
    wtx= np.multiply(x, coord_wts)

    def uniform_to_laplace(u, mu=0, b=1):
        return mu - b * np.sign(u - 0.5) * np.log(1 - 2 * np.abs(u - 0.5))

    laplace_rv = uniform_to_laplace(wtx, mu=0, b=1)
    return laplace_rv

In [ ]:
yiid1_10_laplace=testfun_laplace(iidpts1_10)
yiid1_11_laplace=testfun_laplace(iidpts1_11)
yiid1_12_laplace=testfun_laplace(iidpts1_12)
yiid1_13_laplace=testfun_laplace(iidpts1_13)
yiid1_14_laplace=testfun_laplace(iidpts1_14)

ysob1_10_laplace=testfun_laplace(sobpts1_10)
ysob1_11_laplace=testfun_laplace(sobpts1_11)
ysob1_12_laplace=testfun_laplace(sobpts1_12)
ysob1_13_laplace=testfun_laplace(sobpts1_13)
ysob1_14_laplace=testfun_laplace(sobpts1_14)

In [ ]:
xeval = np.linspace(-4, 8, 500)
print(rmse_laplace(0.1, 2**12, yiid1_10_laplace, kernel_epan, kde, xeval))

In [ ]:
# h_list = np.linspace(0.001, 1, 100)
h_list = 2**np.linspace(-10, 0, 100)
graph_rmse_vs_bandwidth(h_list, rmse_laplace, 2**14, yiid1_14_laplace, kernel_epan, kde, xeval, "RMSE vs. bandwidth, 1-D Laplace, IID pts, n = 2^12, Epan. kernel)")

In [ ]:
h_list = [0.001, 0.002, 0.004, 0.01, 0.02, 0.04, 0.1, 0.2, 0.4, 1]
n_list_power = [10, 11, 12, 13, 14]
pts_list = [yiid1_10_laplace, yiid1_11_laplace, yiid1_12_laplace, yiid1_13_laplace, yiid1_14_laplace]
rmse_heatmap(h_list, n_list_power, rmse_laplace, pts_list, kernel_epan, kde, xeval, "RMSE for 1-D Laplace, IID points, Epan. kernel")

In [ ]:
# graph_optimal_bandwidth_vs_n(h_list, n_list_power, rmse_laplace, pts_list, kernel, kde, xeval, "Optimal h vs. n for exponential distribution, dim = 1, Sobol points")

### Weighted sum of normals

If $X_{1}, X_{2}, ..., X_{d}$ all follow the standard uniform distribution, $X_{i} \sim \text{Unif}(0,1)$, then the random variable 
$$
Y = w_{1} \cdot \Phi^{-1}(X_{1}) + w_{2} \cdot \Phi^{-1}(X_{2}) + ... + w_{d} \cdot \Phi^{-1}(X_{d})
$$ 
follows a normal distribution with expectation $0$ and variance $w_{1}^2 + w_{2}^2 + ... + w_{d}^2$. 

In [ ]:
from scipy.stats import norm 
import numpy as np

In [ ]:
def rmse_sum_normals(h, n, pts, kernel, kde, xeval, weights):
    '''
    returns rmse for a KDE that estimates a random variable that is the weighted sum of uniformly distributed random variables

    h = bandwidth
    n = number of points
    pts = set of iid or ld points
    kernel = kernel function
    kde = kde function
    xeval = set of points that is range of x-axis (e.g., np.linspace(-4, 8, 500))
    weights = a list of numbers that are the weights for the sum of normals
    '''

    sum_errors = 0

    sum_normals = xeval.copy()
    
    # Define location (mean) and scale (standard deviation) parameters
    loc = 0
    scale = np.sqrt(sum(w*w for w in weights))
 
    for i in range(len(sum_normals)):
       sum_normals[i]=norm.pdf(sum_normals[i], loc, scale)
    
    rhoval = kde(kernel, pts, h, xeval)

    for i in range(500):
        sum_errors += (sum_normals[i] - rhoval[i])**2
    
    rmse = np.sqrt((1/n)*sum_errors)
    return rmse

In [ ]:
# def copy_testfun_sum_normals(x, weights):  #takign one uniform rv and transforming into a sum of normals

#     y = np.zeros(x.shape[0])

#     for dim in range(x.shape[1]):
#         normal = weights[dim] * norm.ppf(x[:, dim])
#         for i in range(y.shape[0]):
#             y[i] += normal[i]

#     return y

In [ ]:
def testfun_sum_normals(x, weights):  #takign one uniform rv and transforming into a sum of normals

    y = np.sum(norm.ppf(x)*weights, axis = 1)
    # y = np.sum(x*weights, axis = 1)  # testing out this function

    return y

In [ ]:
## testing out testfun_sum_normals without norm.ppf
# x = np.array([[0.2, 0.3, 0.4], [0.1, 0.2, 0.3]])
# weights = [1.0, 0.5, 0.25]
# print(copy_testfun_sum_normals(x, weights))

In [ ]:
yiid3_10_sum_normals=testfun_sum_normals(iidpts3_10, [1,1,1])
yiid3_11_sum_normals=testfun_sum_normals(iidpts3_11, [1,1,1])
yiid3_12_sum_normals=testfun_sum_normals(iidpts3_12, [1,1,1])
yiid3_13_sum_normals=testfun_sum_normals(iidpts3_13, [1,1,1])
yiid3_14_sum_normals=testfun_sum_normals(iidpts3_14, [1,1,1])

ysob3_10_sum_normals=testfun_sum_normals(sobpts3_10, [1,1,1])
ysob3_11_sum_normals=testfun_sum_normals(sobpts3_11, [1,1,1])
ysob3_12_sum_normals=testfun_sum_normals(sobpts3_12, [1,1,1])
ysob3_13_sum_normals=testfun_sum_normals(sobpts3_13, [1,1,1])
ysob3_14_sum_normals=testfun_sum_normals(sobpts3_14, [1,1,1])

yiid5_10_sum_normals=testfun_sum_normals(iidpts5_10, [1,1,1,1,1])
yiid5_11_sum_normals=testfun_sum_normals(iidpts5_11, [1,1,1,1,1])
yiid5_12_sum_normals=testfun_sum_normals(iidpts5_12, [1,1,1,1,1])
yiid5_13_sum_normals=testfun_sum_normals(iidpts5_13, [1,1,1,1,1])
yiid5_14_sum_normals=testfun_sum_normals(iidpts5_14, [1,1,1,1,1])

ysob5_10_sum_normals=testfun_sum_normals(sobpts5_10, [1,1,1,1,1])
ysob5_11_sum_normals=testfun_sum_normals(sobpts5_11, [1,1,1,1,1])
ysob5_12_sum_normals=testfun_sum_normals(sobpts5_12, [1,1,1,1,1])
ysob5_13_sum_normals=testfun_sum_normals(sobpts5_13, [1,1,1,1,1])
ysob5_14_sum_normals=testfun_sum_normals(sobpts5_14, [1,1,1,1,1])

In [ ]:
xeval = np.linspace(-4, 8, 500)
weights_3 = [1, 1, 1]
weights_5 = [1, 1, 1, 1, 1]
print(rmse_sum_normals(0.1, 2**10, yiid3_10_sum_normals, kernel_epan, kde, xeval, weights_3))

In [ ]:
# h_list = np.linspace(0.001, 1, 100)
h_list = 2**np.linspace(-10, 0, 100)
weighted_graph_rmse_vs_bandwidth(h_list, rmse_sum_normals, 2**12, yiid5_12_sum_normals, kernel_epan, kde, xeval, weights_5, "RMSE vs. bandwidth for sum of 5 Gaussians, IID points, n = 2^12, Epan. kernel")

In [ ]:
h_list = [0.001, 0.002, 0.004, 0.01, 0.02, 0.04, 0.1, 0.2, 0.4, 1]
n_list_power = [10, 11, 12, 13, 14]
pts_list = [yiid5_10_sum_normals, yiid5_11_sum_normals, yiid5_12_sum_normals, yiid5_13_sum_normals, yiid5_14_sum_normals]
weighted_rmse_heatmap(h_list, n_list_power, rmse_sum_normals, pts_list, kernel_epan, kde, xeval, weights_5, "RMSE for sum of 5 Gaussians, IID points, Epan. kernel")

In [ ]:
# weighted_graph_optimal_bandwidth_vs_n(h_list, n_list_power, rmse_sum_normals, ysob3_sum_normals, kernel, kde, xeval, weights_3, "Optimal h vs. n for sum of normals, dim = 3, Sobol points")

### Chi-squared (sum of squared Gaussians)
A chi-square-distributed random variable with $k$ degrees of freedom is defined as the sum of $k$ squared standard normal random variables:
$$
X_{1}, ..., X_{k} \rightarrow Y = \sum_{i=1}^{k} X_{i}^{2} = \chi^{2}(k)
$$
https://statproofbook.github.io/P/chi2-pdf.html

In [ ]:
from scipy.stats import chi2

In [ ]:
def rmse_chi_squared(h, n, pts, kernel, kde, xeval, df):
    '''
    returns rmse for a KDE that estimates a random variable that is the sum of df squared gaussians, i.e., chi-squared with degrees of freedom = df

    h = bandwidth
    n = number of points
    pts = set of iid or ld points
    kernel = kernel function
    kde = kde function
    xeval = set of points that is range of x-axis (e.g., np.linspace(-4, 8, 500))
    df = degrees of freedom (how many squared gaussians you are summing)
    '''

    sum_errors = 0

    chi_squared = xeval.copy()
 
    for i in range(len(chi_squared)):
       chi_squared[i] = chi2.pdf(chi_squared[i], df)
    
    rhoval = kde(kernel, pts, h, xeval)

    for i in range(500):
        sum_errors += (chi_squared[i] - rhoval[i])**2
    
    rmse = np.sqrt((1/n)*sum_errors)
    return rmse

In [ ]:
def testfun_chi_square(x, df):
    y = np.sum((norm.ppf(x))**2, axis = 1)
    return y

In [ ]:
yiid1_10_chi_square=testfun_chi_square(iidpts1_10, 1)
yiid1_11_chi_square=testfun_chi_square(iidpts1_11, 1)
yiid1_12_chi_square=testfun_chi_square(iidpts1_12, 1)
yiid1_13_chi_square=testfun_chi_square(iidpts1_13, 1)
yiid1_14_chi_square=testfun_chi_square(iidpts1_14, 1)

ysob1_10_chi_square=testfun_chi_square(sobpts1_10, 1)
ysob1_11_chi_square=testfun_chi_square(sobpts1_11, 1)
ysob1_12_chi_square=testfun_chi_square(sobpts1_12, 1)
ysob1_13_chi_square=testfun_chi_square(sobpts1_13, 1)
ysob1_14_chi_square=testfun_chi_square(sobpts1_14, 1)

yiid3_10_chi_square=testfun_chi_square(iidpts3_10, 3)
yiid3_11_chi_square=testfun_chi_square(iidpts3_11, 3)
yiid3_12_chi_square=testfun_chi_square(iidpts3_12, 3)
yiid3_13_chi_square=testfun_chi_square(iidpts3_13, 3)
yiid3_14_chi_square=testfun_chi_square(iidpts3_14, 3)

ysob3_10_chi_square=testfun_chi_square(sobpts3_10, 3)
ysob3_11_chi_square=testfun_chi_square(sobpts3_11, 3)
ysob3_12_chi_square=testfun_chi_square(sobpts3_12, 3)
ysob3_13_chi_square=testfun_chi_square(sobpts3_13, 3)
ysob3_14_chi_square=testfun_chi_square(sobpts3_14, 3)

yiid5_10_chi_square=testfun_chi_square(iidpts5_10, 5)
yiid5_11_chi_square=testfun_chi_square(iidpts5_11, 5)
yiid5_12_chi_square=testfun_chi_square(iidpts5_12, 5)
yiid5_13_chi_square=testfun_chi_square(iidpts5_13, 5)
yiid5_14_chi_square=testfun_chi_square(iidpts5_14, 5)

ysob5_10_chi_square=testfun_chi_square(sobpts5_10, 5)
ysob5_11_chi_square=testfun_chi_square(sobpts5_11, 5)
ysob5_12_chi_square=testfun_chi_square(sobpts5_12, 5)
ysob5_13_chi_square=testfun_chi_square(sobpts5_13, 5)
ysob5_14_chi_square=testfun_chi_square(sobpts5_14, 5)

In [ ]:
xeval = np.linspace(-4, 8, 500)
print(rmse_chi_squared(0.1, 2**10, yiid3_10_chi_square, kernel, kde, xeval, df=3))

In [ ]:
# h_list = np.linspace(0.001, 1, 100)
h_list = 2**np.linspace(-10, 0, 100)
weighted_graph_rmse_vs_bandwidth(h_list, rmse_chi_squared, 2**14, ysob5_14_chi_square, kernel_epan, kde, xeval, 5, "RMSE vs. bandwidth for chi-square, df = 5: Sobol points, n = 2^12, Epan. kernel")

In [ ]:
h_list = [0.001, 0.002, 0.004, 0.01, 0.02, 0.04, 0.1, 0.2, 0.4, 1]
n_list_power = [10, 11, 12, 13, 14]
pts_list = [ysob5_10_chi_square, ysob5_11_chi_square, ysob5_12_chi_square, ysob5_13_chi_square, ysob5_14_chi_square]
weighted_rmse_heatmap(h_list, n_list_power, rmse_chi_squared, pts_list, kernel_epan, kde, xeval, 5, "RMSE for chi-square, df = 5, Epan. kernel, Sobol points")

## Using dictionaries for RMSE

In [ ]:
import scipy as sc

In [ ]:
uniform_1 = {
    "f" : lambda x: x[:,0],  #just take the first coordinate
    "density" : lambda y,d : np.where((y >= 0) & (y <= 1),1.,0.),
    "y_domain" : lambda d : np.array([-0.1,1.1]),
    "name" : lambda d=1 : "One-Dimensional Uniform"}
sum_d_Gauss = {
    "f" : lambda x: np.sum(sc.stats.norm.ppf(x),axis = 1),  #sum of d standard Gaussians is a Gaussian with variance 
    "density" : lambda y,d : sc.stats.norm.pdf(y,scale = np.sqrt(d)),
     "y_domain" : lambda d : np.array([-3*np.sqrt(d),3*np.sqrt(d)]),
    "name" : lambda d : f"Sum of {d:.0f} Gaussians"}
print(uniform_1["density"](np.array([0.5,1,1.5]),1))
print(sum_d_Gauss["density"](0,2))
ex_densities = [uniform_1,sum_d_Gauss]
d = 3
for ii in range(len(ex_densities)):
    fig,ax = plt.subplots()
    domain = ex_densities[ii]["y_domain"](d)
    yplot = np.linspace(domain[0],domain[1],500)
    rhoplot = ex_densities[ii]["density"](yplot,d)
    ax.plot(yplot, rhoplot),
    ax.set_title(ex_densities[ii]["name"](d))
    ax.set_xlabel("$y$")
    ax.set_ylabel("probability density")

In [ ]:
rmse = lambda rhoval, pdf, n : np.sqrt((1/n)*(np.sum(np.square(pdf-rhoval))))

In [ ]:
uniform_1 = {
    "f" : lambda x: x[:,0],  #just take the first coordinate
    "density" : lambda y,d : np.where((y >= 0) & (y <= 1),1.,0.),
    "y_domain" : lambda d : np.array([-0.1,1.1]),
    "name" : lambda d=1 : "One-Dimensional Uniform"}

In [ ]:
gaussian_1 = { # standard Gaussian
    "f" : lambda x: norm.ppf(x), 
    "density" : lambda y,d : norm.pdf(y, loc=0, scale=1),
    "y_domain" : lambda d : np.array([-3.1,3.1]),
    "name" : lambda d=1 : "One-Dimensional Standard Gaussian"}

In [ ]:
triangular_1 = { # triangular = sum of 2 uniform variables, min = 0, max = 2, mode = 1
    "f" : lambda x: np.where(x < 0.5,
                        0 + np.sqrt(x * (2) * (1)),
                        2 - np.sqrt((1 - x) * (2) * (1))), 
    "density" : lambda y,d : np.where((0 < y) & (y < 1), y, 
                                      np.where((1 <= y) & (y < 2), 2 - y, 
                                               0)),
    "y_domain" : lambda d : np.array([-0.1, 2.1]),
    "name" : lambda d=1 : "One-Dimensional Sum of 2 Uniforms (Triangular)"}

In [ ]:
# from scipy.stats import expon

exp_1 = { # exponential, lambda = 1
    "f" : lambda x: -np.log(x), 
    "density" : lambda y,d : expon.pdf(y),
    "y_domain" : lambda d : np.array([-0.1,4.1]),
    "name" : lambda d=1 : "One-Dimensional Exponential (lambda = 1)"}

In [ ]:
laplace_1 = { # laplace, where mu = 0 and b = 1
    "f" : lambda x: 0 - 1 * np.sign(x - 0.5) * np.log(1 - 2 * np.abs(x - 0.5)), 
    "density" : lambda y,d : stats.laplace(0, 1).pdf(y),
    "y_domain" : lambda d : np.array([-4.1,4.1]),
    "name" : lambda d=1 : "One-Dimensional Laplace (mu = 0, b = 1)"}

In [ ]:
densities = [uniform_1, gaussian_1, triangular_1, exp_1, laplace_1]
d = 1
for ii in range(len(densities)):
    fig,ax = plt.subplots()
    domain = densities[ii]["y_domain"](d)
    yplot = np.linspace(domain[0],domain[1],500)
    rhoplot = densities[ii]["density"](yplot,d)
    ax.plot(yplot, rhoplot),
    ax.set_title(densities[ii]["name"](d))
    ax.set_xlabel("$y$")
    ax.set_ylabel("probability density")

In [ ]:
weight_sum_d_Gauss = {
    "f" : lambda x, weights: np.sum(norm.ppf(x)*weights, axis = 1),  #sum of d standard Gaussians with weights 
    "density" : lambda y, weights : sc.stats.norm.pdf(y,scale = np.sqrt(sum(w*w for w in weights))),
     "y_domain" : lambda weights : np.array([-3*np.sqrt(sum(w*w for w in weights)),3*np.sqrt(sum(w*w for w in weights))]),
    "name" : lambda d : f"Weighted Sum of {d:.0f} Gaussians"}

In [ ]:
weighted_densities = [weight_sum_d_Gauss]
d = 3
weights_3 = [1,1,1]
for ii in range(len(weighted_densities)):
    fig,ax = plt.subplots()
    domain = weighted_densities[ii]["y_domain"](weights_3)
    yplot = np.linspace(domain[0],domain[1],500)
    rhoplot = weighted_densities[ii]["density"](yplot,weights_3)
    ax.plot(yplot, rhoplot),
    ax.set_title(weighted_densities[ii]["name"](d))
    ax.set_xlabel("$y$")
    ax.set_ylabel("probability density")

In [ ]:
def new_graph_rmse_vs_bandwidth(distn_dict, h_list, n, pts, kernel, kde, xeval, graph_title):
    '''
    generates a graph of rmse vs. log-scaled bandwidth
    
    distn_dict = dictionary that defines the distribution we are estimating
    h_list = list of bandwidths to be tested (log-scaled bandwidth on x-axis)
    n = number of points
    pts = set of untransformed points (IID or LD)
    kernel = kernel function
    kde = kernel density estimator function
    xeval = set of points that is range of x-axis (e.g., np.linspace(-4, 8, 500))
    graph_title = a string that will become title of graph
    '''

    rmse_list = []
    for h in h_list:
        rhoval = kde(kernel, distn_dict['f'](pts), h, xeval)
        rmse_list.append(rmse(rhoval, distn_dict['density'](xeval, 1), n))

    min_index = rmse_list.index(min(rmse_list))
    print("Minimum rmse = " 
        + str(rmse_list[min_index]) 
        + "\nand corresponding bandwidth is " 
        + str(h_list[min_index])
        + ".\nLog of optimal bandwidth is "
        + str(np.log(h_list[min_index])))
    
    fig, ax = plt.subplots()
    ax.plot(h_list, rmse_list)
    ax.set_title(graph_title)
    ax.set_xlabel('Log-Scaled Bandwidth')
    ax.set_ylabel('RMSE')
    ax.set_xscale('log')

In [ ]:
def weighted_new_graph_rmse_vs_bandwidth(distn_dict, h_list, n, pts, kernel, kde, xeval, weights, graph_title):
    '''
    generates a graph of rmse vs. log-scaled bandwidth
    
    distn_dict = dictionary that defines the distribution we are estimating
    h_list = list of bandwidths to be tested (log-scaled bandwidth on x-axis)
    n = number of points
    pts = set of untransformed points (IID or LD)
    kernel = kernel function
    kde = kernel density estimator function
    xeval = set of points that is range of x-axis (e.g., np.linspace(-4, 8, 500))
    weights = list of weights
    graph_title = a string that will become title of graph
    '''

    rmse_list = []
    for h in h_list:
        rhoval = kde(kernel, distn_dict['f'](pts, weights), h, xeval)
        rmse_list.append(rmse(rhoval, distn_dict['density'](xeval, weights), n))

    min_index = rmse_list.index(min(rmse_list))
    print("Minimum rmse = " 
        + str(rmse_list[min_index]) 
        + "\nand corresponding bandwidth is " 
        + str(h_list[min_index])
        + ".\nLog of optimal bandwidth is "
        + str(np.log(h_list[min_index])))
    
    fig, ax = plt.subplots()
    ax.plot(h_list, rmse_list)
    ax.set_title(graph_title)
    ax.set_xlabel('Log-Scaled Bandwidth')
    ax.set_ylabel('RMSE')
    ax.set_xscale('log')

In [ ]:
xeval = np.linspace(-4, 8, 500)
h_list = 2**np.linspace(-10, 0, 100)
new_graph_rmse_vs_bandwidth(exp_1, h_list, 2**12, iidpts1, kernel, kde, xeval, "RMSE vs. bandwidth for IID pts: n = 2^12, dim = 1")

In [ ]:
xeval = np.linspace(-4, 8, 500)
h_list = 2**np.linspace(-10, 0, 100)
weights_1 = [1]
weights_3 = [1,1,1]
weighted_new_graph_rmse_vs_bandwidth(weight_sum_d_Gauss, h_list, 2**12, iidpts1, kernel, kde, xeval, weights_1, "RMSE vs. bandwidth for IID pts: n = 2^12, dim = 1")

## Profiling code

In [ ]:
# import matplotlib.pyplot as plt
# import math
# import numpy as np
# import scipy as sp
# import qmcpy as qp
# from scipy import stats
# from copy import deepcopy
# import sympy as sy
# import sympy.stats
# import pprofile
# import sys

In [ ]:
# ## defining functions
# # QMC Point Generators
# def gen_iid_ld_pts(dimen = 3, n = 2**12):
#     """
#     Generate IID, Sobol, Lattic, DNB2, and Halton sample points,
#     based on dimension and smaple size.
#     """
#     coord_wts = 2.0**(-np.array(range(0, dimen)))
#     iidpts = qp.IIDStdUniform(dimen).gen_samples(n)
#     sobpts = qp.Sobol(dimen).gen_samples(n)
#     latticepts = qp.Lattice(dimen).gen_samples(n)
#     dnetb2pts = qp.DigitalNetB2(dimen).gen_samples(n)
#     haltonpts = qp.Halton(dimen).gen_samples(n)
#     return (iidpts, sobpts, latticepts, dnetb2pts, haltonpts)

# # Kernels
# # hardcoding hermite kernel coefficients
# def hardcode_hermite_kernel(r):
#     '''
#     returns hermite coefficients for r = 0, 1, or 2
#     band_width = the kernel band width, often denoted h; a smaller bandwidth means a peakier kernel
#     r = number of terms in our kernel
#     '''
#     if r == 0:
#         return sy.Matrix([[1/(math.pi)**0.5]])
#     elif r == 1:
#         return sy.Matrix([[1/(math.pi)**0.5, -1/(4*(math.pi)**0.5)]])
#     elif r == 2:
#         return sy.Matrix([[1/(math.pi)**0.5, -1/(4*(math.pi)**0.5), 1/(32*(math.pi)**0.5)]])
#     else:
#         raise ValueError("Invalid input. r must be 0, 1, or 2.")

# # hardcoded: define hermite kernel for r = 0 (corresponds to r = 1 in the original case)
# def hardcode_hermite_kernel_weight(y, r = 0):
#     '''
#     r = number of terms in our kernel
#     '''
#     coef = np.array((hardcode_hermite_kernel(r = r)))
#     # print(coef)
#     # poly = sp.special.hermite(0)
#     # print(poly)
#     k = sp.special.hermite(0)(y) * coef[0][0] #initialize a vector of kernel values
#     # print(k)
#     for ii in range(0, r):
#         k += sp.special.hermite(2*ii)(y) * coef[0][ii] #add the additional terms
#     k *= np.exp(-y*y/2) #normalizing weight for Hermite functions #form the isotropic kernel and insert the bandwidth dependency
#     return k

# # hardcoding hermite kernel coefficients without sympy
# def hardcode_hermite_kernel_simple(r):
#     '''
#     returns hermite coefficients for r = 0, 1, or 2
#     band_width = the kernel band width, often denoted h; a smaller bandwidth means a peakier kernel
#     r = number of terms in our kernel
#     '''
#     if r==0:
#         return np.matrix([[1/(math.pi)**0.5]])
#     elif r==1:
#         return np.matrix([[1/(math.pi)**0.5, -1/(4*(math.pi)**0.5)]])
#     elif r==2:
#         return np.matrix([[1/(math.pi)**0.5, -1/(4*(math.pi)**0.5), 1/(32*(math.pi)**0.5)]])
#     else:
#         raise ValueError("Invalid input. r must be 0, 1, or 2.")
    
# # using simplified np.matrix for coefficients, instead of sympy.matrix
# def hardcode_hermite_kernel_weight_v2(y, r = 0):
#     '''
#     r = number of terms in our kernel
#     '''
#     coef = np.array((hardcode_hermite_kernel_simple(r = r)))
#     # print(coef)
#     # poly = sp.special.hermite(0)
#     # print(poly)
#     k = sp.special.hermite(0)(y) * coef[0][0] #initialize a vector of kernel values
#     # print(k)
#     for ii in range(0, r):
#         k += sp.special.hermite(2*ii)(y) * coef[0][ii] #add the additional terms
#     k *= np.exp(-y*y/2) #normalizing weight for Hermite functions #form the isotropic kernel and insert the bandwidth dependency
#     return k
    
# # hardcoded without scipy: define hermite kernel for r = 0 (corresponds to r = 1 in the original case)
# # for some reason, not returning same values as original kernel
# def simple_hardcode_hermite_kernel_weight(y, r = 0):
#     '''
#     r = number of terms in our kernel
#     '''
#     coef = np.array((hardcode_hermite_kernel(r = r)))
#     hermite_polys = [sp.poly1d([1.]), sp.poly1d([4.,  0., -2.]), sp.poly1d([ 1.6000000e+01,  0.0000000e+00, -4.8000000e+01, -8.8817842e-16,
#         1.2000000e+01])]
#     # print(coef)
#     # poly = sp.special.hermite(0)
#     # print(poly)
#     k = sp.poly1d([1.])(y)*coef[0][0] #initialize a vector of kernel values
#     # print(k)
#     for ii in range(0,r):
#         k += hermite_polys[r](y)*coef[0][ii] #add the additional terms
#     k*=np.exp(-y*y/2) #normalizing weight for Hermite functions #form the isotropic kernel and insert the bandwidth dependency
#     return k

# # PDFs
# # Unamed distribution
# def testfun(x):
#     return 10 * np.exp(-x) * np.sin(np.pi*x)

# # Guassian distribution
# def guass_distr(x, coord_wts = 1): # function f(x) defines the random variable
#     wtx = np.multiply(x, coord_wts)
#     y = 10 * np.exp(-wtx.sum(1)) * np.sin(np.pi * wtx.sum(1))
#     #y = x[:,0] # if x is uniform, then y is uniform
#     return y

# # KDEs
# def kde_pt(kernel, ypts, bandwidth, yeval):
#     return np.mean(kernel((yeval-ypts) / bandwidth)) / bandwidth

# # def estimated_pts(x):
# #     return kde_pt(hardcode_hermite_kernel_weight, pts, h, x)

# # def squared_difference_pts(x):
# #     return (hardcode_hermite_kernel_weight - estimated_pts(x))**2

# # Integration operation from SciPy
# def integration_test(h, pts, kde, kernel, true_function):
#         def estimated_pts(x):
#             return kde(kernel, pts, h, x)
#         def squared_difference_pts(x):
#             return (true_function(x) - estimated_pts(x))**2
#         mise, error = sp.integrate.quad(squared_difference_pts, -4, 8) # This seems to be taking the longest. Will investigate.
#         return (mise, error)

# # All operations
# def optimal_h(h_space, pts, kde, kernel, true_function):
#     '''
#     returns optimal bandwidth and lowest value of MISE
#     h_space: a set of bandwidths to be tested (e.g., np.linspace(0.001, 3, 100))
#     pts: set of randomly generated points (IID or LD)
#     kde: kde function
#     kernel: function that defines kernel
#     true_function: function that we are trying to estimate (e.g., f(x), laplace, uniform, exponential, etc.)
#     '''
#     h_optimal = 0
#     lowest_mise = np.inf
#     for h in h_space:
#         def estimated_pts(x):
#             return kde(kernel, pts, h, x)
#         def squared_difference_pts(x):
#             return (true_function(x) - estimated_pts(x))**2
#         mise, error = sp.integrate.quad(squared_difference_pts, -4, 8) # TODO: Fix this interval of integration so that it makes sense. Should probably be parameterized as well.
#         if mise < lowest_mise:
#             lowest_mise = mise
#             h_optimal = h

#     return(lowest_mise, h_optimal)

In [ ]:
# # Functions to test
# # Alternatives to SciPy's integration

# iidpts1, sobpts1, latticepts1, dnetb2pts1, haltonpts1 = gen_iid_ld_pts(dimen = 1, n = 2**12)
# yiid1 = guass_distr(iidpts1)
# test_option = '6'

# profiler = pprofile.Profile()
# with profiler:
#     if test_option in ('optimal_h' ,'0'):
#         result = optimal_h(np.linspace(0.001, 2, 2), yiid1, kde_pt, hardcode_hermite_kernel_weight, testfun)
#     elif test_option in ('kde_pt', '1'):
#         result = kde_pt(hardcode_hermite_kernel_weight, yiid1, 0.5, 2.0)
#     # elif test_option == 'hardcode_hermite_kernel_weight':
#     #     # result = hardcode_hermite_kernel_weight(y, r)
#     elif test_option in ('hardcode_hermite_kernel', '2'):
#         r_number = int(2)
#         print(hardcode_hermite_kernel(r_number))
#     elif test_option in ('hardcode_hermite_kernel', '3'):
#         r_number = int(2)
#         result = hardcode_hermite_kernel_weight(y=yiid1, r=r_number)
#     elif test_option in ('integration_test', '4'):
#         mise, error = integration_test(0.1, yiid1, kde_pt, hardcode_hermite_kernel_weight, testfun)
#     elif test_option in ('hardcode_hermite_kernel_simple', '5'):
#         print(hardcode_hermite_kernel_simple(2)) # testing when r = 2
#     elif test_option in ('simple_hardcode_hermite_kernel_weight', '6'):
#         print(simple_hardcode_hermite_kernel_weight(y=yiid1, r=2))
#     elif test_option in ('hardcode_hermite_kernel_weight_v2', '7'):
#         result = hardcode_hermite_kernel_weight_v2(y=yiid1, r=2)
#     else:
#         raise Exception

# profiler.print_stats()

In [ ]:
#testing